# Successive Small Gains - An Equity Market Analysis

Equity markets consist of some of the most studied data in the world. With millions invested into research, and computational tools beyond what's accessible to the average individual, it's not hard to see that an individual investor doing their own analysis will have a tough time outperforming. Currently, I see two paths to success:

1. **Find a strategy the institutions haven't:** while possible, this is unlikely to happen considering the disparity in resources spent on the problem. You'd have to be incredibly lucky, and even still it's probably only a matter of time before an institution catches on, at which point any alpha is lost.


2. **Find a strategy that institutions can't implement:** while the large amount of resources gives institutions ample opportunity, they also impose limitations. In particular, investing large amounts of capital has an impact on the market by moving equity prices and as a result signalling the rest of the market. This effect is especially pronounced in smaller caps. Moving large blocks of stock can be difficult and imprecise. 

The second point presents an opportunity, and is the basis for this project. 

Imagine a stock with a 500m market cap, that has been choppy for the past two years, moving cyclically between \\$4.50 and \\$5.50. An institution sets a buy price at \\$5.00 and sell at \\$5.10, hoping to lock in a 2\% gain every trade. Putting in an order to buy 100k shares at \\$5.00 will likely settle in blocks. Depending on the liquidity of the stock, executing such a large order will push up the stock price. In all likelihood, some blocks will settle at \\$5.00, some at \\$5.01... etc. Selling those 100k shares will have the opposite effect, pushing down the price during execution of the order. The result is far from the 2\% expected return (and even has the potential to be negative depending on how much the order moves the stock price). Buying 100 shares; however, will likely settle in a single block, and thus face no such issues. 



In [6]:
# Import packages
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
from datetime import date
from pandas.tseries.offsets import QuarterEnd

# A new way to filter stocks? Eliminate based on max deviation from mean

In [54]:
#set start date. Backtest will check 1yr back, and calculate returns 1yr forward 
model_start_date = datetime.date(2020,3,1) 

In [49]:
def min_var_filter(ticker, var=0.25, model_start_date=model_start_date, period=365):
    
    """
    Find if price ever moves var% away from mean over period (default starting from today looking back 1yr).
        if not enough data, set var to -2
        if enough data, but max/min price too high/low set var to -1.
        else find var to make sure price isn't TOO stable

    """
    #set date range
    start_date = model_start_date - datetime.timedelta(period) 
    end_date = model_start_date
    
    #retrieve data
    
    stock_data = yf.download(ticker, start = start_date, end = end_date, interval = "1d")
    closing_price_data = np.array(stock_data["Close"])
    
    #need try/catch for tickers that fail to download (usually Data doesn't exist for given date range)
    try:
        avg_close = np.mean(closing_price_data)
        max_close = np.max(closing_price_data)
        min_close = np.min(closing_price_data)
    except (ValueError):
        pv = -2
        return {"PV": pv}
    
    upper_bound = (1+var)*avg_close
    lower_bound = (1-var)*avg_close
    
    #check for recent IPO (assume trading days/yr, 10 day buffer)
    if (len(closing_price_data) < (period*250/365 - 10)):
        pv = -2    
    #check prices
    #if price stays within bounds, calculate variability
    elif (max_close > upper_bound) or (min_close < lower_bound):
        pv = -1
    else:         
        closing_std_dev = np.std(closing_price_data) 
        variability = closing_std_dev / avg_close #price adjusted deviation
        pv = variability
    
    return {"PV": pv}


In [ ]:
#TEST
def back_calculate_returns(list_of_stocks, start_date, end_date):
    """
    Print the optimal buy and sell price of each stock in the given list over the given range
    Also gives total % returns over that period
    """
    
    returns = {}
    
    # need to add these globals since vectorizing, otherwise the program complains that it's calling  
    #.  variables before they're defined
    global owned
    global max_profit
    global best_sell
    global best_buy

    for i, stock in enumerate(list_of_stocks):
        info = yf.download(stock, start=start_date, end=end_date)[["High", "Low"]]

        info = info.round(2)

        owned = False
        max_profit = 0.
        best_buy = 0.
        best_sell = 0.

        min_low = np.min(info["Low"])
        max_high = np.max(info["High"])

        #setting increment at roughtly 1/2% of min_low
        if (min_low < 5):
            increment = 0.02
        elif (min_low < 10):
            increment = 0.05
        elif (min_low < 25):
            increment = 0.12
        elif (min_low < 50):
            increment = 0.25
        elif (min_low < 75):
            increment = 0.35
        else:
            increment = 0.5
            
        buy_prices = np.arange(min_low, max_high, increment)

        def buy(buy_price):

            global owned
            global max_profit
            global best_sell
            global best_buy

            sell_prices = np.arange(buy_price, max_high, increment)

            def sell(sell_price):
                global owned
                global max_profit
                global best_sell
                global best_buy

                profit = 0.

                for index, row in info.iterrows():
                    if (owned):
                        if (row["High"] >= sell_price):
                            profit += (sell_price - buy_price)
                            owned = False
                    else:
                        if (row["Low"] <= buy_price):
                            owned = True

                    if (profit > max_profit):
                        max_profit = round(profit, 2)
                        best_sell = round(sell_price, 2)
                        best_buy = round(buy_price, 2)

            sell_vec = np.vectorize(sell)
            sell_vec(sell_prices)

        buy_vec = np.vectorize(buy)
        buy_vec(buy_prices)

        percent_return = max_profit / best_buy * 100 

        print(f"The profit for {stock} is {max_profit} at buy price: {best_buy} and sell price: {best_sell}")
        print(f"That equates to a {percent_return}% return")
        
        #only include stocks with more than 2 buys and sells to make optimal profits in the returned dict
        if (max_profit > (best_sell - best_buy)*2):
            returns.update({stock : {"buy_price" : best_buy, "sell_price" : best_sell, 
                "min_low" : min_low, "max_high" : max_high}})           

    return returns


In [81]:
def back_calculate_returns(list_of_stocks, start_date, end_date):
    """
    Print the optimal buy and sell price of each stock in the given list over the given range
    Also gives total % returns over that period
    """
    
    returns = {}
    
    # need to add these globals since vectorizing, otherwise the program complains that it's calling  
    #.  variables before they're defined
    global owned
    global max_profit
    global best_sell
    global best_buy

    for i, stock in enumerate(list_of_stocks):
        info = yf.download(stock, start=start_date, end=end_date)[["High", "Low"]]

        info = info.round(2)

        owned = False
        max_profit = 0.
        best_buy = 0.
        best_sell = 0.

        min_low = np.min(info["Low"])
        max_high = np.max(info["High"])

        # need this to prevent ValueError from weird values
        if min_low > 0 and max_high > min_low:
            # incrementing at 1% of buy price to reduce runtime
            # ideally would do 0.01 for all but that's probably more trouble than it's worth at high prices
            # adding 0.0001 to make sure there's no 0 vals
            increment = 0.05
            
        buy_prices = np.arange(min_low, max_high, increment)

        def buy(buy_price):

            global owned
            global max_profit
            global best_sell
            global best_buy

            sell_prices = np.arange(buy_price, max_high, increment)

            def sell(sell_price):
                global owned
                global max_profit
                global best_sell
                global best_buy

                profit = 0.

                for index, row in info.iterrows():
                    if (owned):
                        if (row["High"] >= sell_price):
                            profit += (sell_price - buy_price)
                            owned = False
                    else:
                        if (row["Low"] <= buy_price):
                            owned = True

                    if (profit > max_profit):
                        max_profit = round(profit, 2)
                        best_sell = round(sell_price, 2)
                        best_buy = round(buy_price, 2)

            sell_vec = np.vectorize(sell)
            sell_vec(sell_prices)

        buy_vec = np.vectorize(buy)
        buy_vec(buy_prices)

        percent_return = max_profit / best_buy * 100 

        print(f"The profit for {stock} is {max_profit} at buy price: {best_buy} and sell price: {best_sell}")
        print(f"That equates to a {percent_return}% return")
        
        #only include stocks with more than 2 buys and sells to make optimal profits in the returned dict
        if (max_profit > (best_sell - best_buy)*2):
            returns.update({stock : {"buy_price" : best_buy, "sell_price" : best_sell, 
                "min_low" : min_low, "max_high" : max_high}})           

    return returns


In [15]:
def front_calculate_returns(dict_of_stocks, start_date, end_date):
    """
    Front calculate returns using given dict with key: ticker vals: buy/sell prices, backwards looking 
       max high/min low 
    Sell out if price goes above max high/min low
    """
    
    global owned
    
    avg_returns_list = []

    for ticker in dict_of_stocks:
        
        prices = dict_of_stocks[ticker]
        buy_price = prices["buy_price"]
        sell_price = prices["sell_price"]
        min_low = prices["min_low"]
        max_high = prices["max_high"]
        
        info = yf.download(ticker, start=start_date, end=end_date)[["High", "Low"]]

        info = info.round(2)

        owned = False
        
        profit = 0.

        for index, row in info.iterrows():
            #break if you leave the bounds of the training data
            if (row["Low"] < min_low):
                if (owned):
                    profit += row["Low"] - buy_price
                break
            #removed exit on upside out of bounds
                
            if (owned):
                if (row["High"] >= sell_price):
                    profit += (sell_price - buy_price)
                    owned = False
            else:
                if (row["Low"] <= buy_price):
                    owned = True

        percent_return = profit / buy_price * 100

        print(f"The profit for {ticker} is {profit}")
        print(f"That equates to a {percent_return}% return")
        
        avg_returns_list.append(percent_return)
    
    equal_weight_avg_return = np.mean(avg_returns_list)

    print (f"The equal weight average returns are: {equal_weight_avg_return}")
    
    return equal_weight_avg_return

In [19]:
#using present day tickers may bias the model as companies that have since been delisted aren't included. 
# assuming that number is small, and the effect is negligible, but it's something that could be addressed later 
# this does, however, limit how far back I can test without updating the ticker list 
# even starting in 2022 already a lot of stock data not found (newer IPOs)
tickers = pd.read_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-ma' \
                      'rket-analysis/tickers.csv')

tickers = tickers.rename(columns = {"A": "Ticker"})

In [55]:
#all in one instead of breaking it up 1000 at a time

tickers['PV'] = tickers['Ticker'].apply(min_var_filter)
tickers_df = pd.concat([tickers[['Ticker']], tickers['PV'].apply(pd.Series)], axis=1)
tickers_df_filtered = tickers_df[tickers_df["PV"].between(left = 0.03, right = 0.05)]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAC: Data doesn't exist for startDate = 1583038800, endDate = 1614574800
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ZING: Data doesn't exist for startDate = 1583038800, endDate = 1614574800
[*********************100%***********************]  1 of 1 completed


/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ZT: Data doesn't exist for startDate = 1583038800, endDate = 1614574800
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


[*********************100%***********************]  1 of 1 completed


In [56]:
best_stocks = tickers_df_filtered["Ticker"]

start_date = model_start_date - datetime.timedelta(365) #1yr back
end_date = model_start_date

stock_dict = back_calculate_returns(best_stocks, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for AFB is 1.64 at buy price: 12.65 and sell price: 14.29
That equates to a 12.964426877470355% return
[*********************100%***********************]  1 of 1 completed
The profit for AWR is 28.36 at buy price: 67.52 and sell price: 95.88
That equates to a 42.002369668246445% return
[*********************100%***********************]  1 of 1 completed
The profit for BCE is 6.5 at buy price: 46.83 and sell price: 49.0
That equates to a 13.879991458466796% return
[*********************100%***********************]  1 of 1 completed
The profit for CHT is 4.03 at buy price: 35.29 and sell price: 36.63
That equates to a 11.419665627656562% return
[*********************100%***********************]  1 of 1 completed
The profit for CPB is 14.36 at buy price: 35.03 and sell price: 49.39
That equates to a 40.99343419925778% return
[*********************100%***********************]  1 of 1 completed
The profit for DL

In [57]:
start_date = model_start_date 
end_date = model_start_date + datetime.timedelta(365) #1yr forward now

#front_calculate_returns(stock_dict, start_date, end_date)

In [17]:
#using 0.2var, 1yr fwd 1yr back
_2021_03_01_returns = front_calculate_returns(stock_dict, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for AWR is 4.550000000000011
That equates to a 6.076388888888904% return
[*********************100%***********************]  1 of 1 completed
The profit for CPB is -6.489999999999995
That equates to a -13.867521367521357% return
[*********************100%***********************]  1 of 1 completed
The profit for DLY is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for HRL is 4.68
That equates to a 10.167282207256138% return
[*********************100%***********************]  1 of 1 completed
The profit for KR is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for MMD is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for MRK is 20.549999999999983
That equates to a 28.454721683744093% return
[********************

In [23]:
#using 0.2var, 1yr fwd 1yr back
_2020_03_01_returns = front_calculate_returns(stock_dict, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for CHT is 1.3400000000000034
That equates to a 3.797109662794002% return
[*********************100%***********************]  1 of 1 completed
The profit for HRL is 4.07
That equates to a 10.091743119266056% return
[*********************100%***********************]  1 of 1 completed
The profit for MRK is -11.760000000000005
That equates to a -15.512465373961227% return
[*********************100%***********************]  1 of 1 completed
The profit for MUI is -1.4299999999999997
That equates to a -10.295176385889127% return
[*********************100%***********************]  1 of 1 completed
The profit for MYN is -0.7900000000000009
That equates to a -6.210691823899378% return
[*********************100%***********************]  1 of 1 completed
The profit for SJM is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for VGM is -1.33
That equates 

In [31]:
#using 0.15var, 1yr fwd 1yr back
_2021_03_01_returns_15var = front_calculate_returns(stock_dict, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for CPB is -6.489999999999995
That equates to a -13.867521367521357% return
[*********************100%***********************]  1 of 1 completed
The profit for DLY is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for ARTNA is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for ATRI is 141.74999999999955
That equates to a 22.123893805309663% return
The equal weight average returns are: 2.0640931094470765


In [36]:
#using 0.15var, 1yr fwd 1yr back
_2020_03_01_returns_15var = front_calculate_returns(stock_dict, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for CHT is 1.3400000000000034
That equates to a 3.797109662794002% return
[*********************100%***********************]  1 of 1 completed
The profit for ARTNA is -1.5700000000000003
That equates to a -4.52188940092166% return
The equal weight average returns are: -0.362389869063829


### Too few stocks using 0.15 var.

In [42]:
#using 0.175var, 1yr fwd 1yr back
_2021_03_01_returns_175var = front_calculate_returns(stock_dict, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for CPB is -6.489999999999995
That equates to a -13.867521367521357% return
[*********************100%***********************]  1 of 1 completed
The profit for DLY is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for HRL is 4.68
That equates to a 10.167282207256138% return
[*********************100%***********************]  1 of 1 completed
The profit for KR is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for MRK is 20.549999999999983
That equates to a 28.454721683744093% return
[*********************100%***********************]  1 of 1 completed
The profit for SJM is 5.519999999999996
That equates to a 4.884523493496147% return
[*********************100%***********************]  1 of 1 completed
The profit for ARTNA is 0.0
That equates to a 0.0% return
[******************

In [47]:
#using 0.175var, 1yr fwd 1yr back
_2020_03_01_returns_15var = front_calculate_returns(stock_dict, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for CHT is 1.3400000000000034
That equates to a 3.797109662794002% return
[*********************100%***********************]  1 of 1 completed
The profit for HRL is 4.07
That equates to a 10.091743119266056% return
[*********************100%***********************]  1 of 1 completed
The profit for MRK is -11.760000000000005
That equates to a -15.512465373961227% return
[*********************100%***********************]  1 of 1 completed
The profit for MUI is -1.4299999999999997
That equates to a -10.295176385889127% return
[*********************100%***********************]  1 of 1 completed
The profit for SJM is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for ARTNA is -1.5700000000000003
That equates to a -4.52188940092166% return
The equal weight average returns are: -2.7401130631186597


In [53]:
#using 0.25var, 1yr fwd 1yr back
_2021_03_01_returns_175var = front_calculate_returns(stock_dict, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for AWR is 4.550000000000011
That equates to a 6.076388888888904% return
[*********************100%***********************]  1 of 1 completed
The profit for CPB is -6.489999999999995
That equates to a -13.867521367521357% return
[*********************100%***********************]  1 of 1 completed
The profit for DLY is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for FINS is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for FLO is 1.5700000000000003
That equates to a 7.211759301791458% return
[*********************100%***********************]  1 of 1 completed
The profit for GIM is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for HRL is 4.68
That equates to a 10.167282207256138% return
[*******************

In [58]:
#using 0.25var, 1yr fwd 1yr back
_2020_03_01_returns_175var = front_calculate_returns(stock_dict, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for BCE is -4.939999999999998
That equates to a -10.548793508434759% return
[*********************100%***********************]  1 of 1 completed
The profit for CHT is 1.3400000000000034
That equates to a 3.797109662794002% return
[*********************100%***********************]  1 of 1 completed
The profit for FINS is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for HRL is 4.07
That equates to a 10.091743119266056% return
[*********************100%***********************]  1 of 1 completed
The profit for IQI is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for MRK is -11.760000000000005
That equates to a -15.512465373961227% return
[*********************100%***********************]  1 of 1 completed
The profit for MUI is -1.4299999999999997
That equates to a -10.29517638

## Single method: 

In [87]:
def cumulative_returns_calc(dates, var, back_days, front_days):
    
    """
    Use each given date to find new stocks (rebalance) 
    var sets how tight we want to filter stocks 
    back_days is how far back-looking we want to check
    front_days is how for forward looking (should match time between rebalancing days)
    
    **Can adjust function to make transition between quarters more seemless. Right now undefined what happens at \
    at the end of each period (stocks not picked for the next period just disappear rn. need to be sold)
    """
    
    returns = []
    
    for date_str in dates:
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()
        model_start_date = date 

        tickers = pd.read_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-ma' \
                              'rket-analysis/tickers.csv')

        tickers = tickers.rename(columns = {"A": "Ticker"})

        tickers['PV'] = tickers['Ticker'].apply(lambda x: min_var_filter(x, var=var, period=back_days))
        tickers_df = pd.concat([tickers[['Ticker']], tickers['PV'].apply(pd.Series)], axis=1)
        tickers_df_filtered = tickers_df[tickers_df["PV"].between(left = 0.03, right = 0.05)]

        best_stocks = tickers_df_filtered["Ticker"]

        start_date = model_start_date - datetime.timedelta(back_days) 
        end_date = model_start_date

        stock_dict = back_calculate_returns(best_stocks, start_date, end_date)

        start_date = model_start_date 
        end_date = model_start_date + datetime.timedelta(front_days) #1yr forward now

        period_returns = front_calculate_returns(stock_dict, start_date, end_date)
        
        returns.append(period_returns)
    
    return returns

In [72]:
test_dates = pd.period_range(start='2017-12-31', end='2022-12-31', freq='Q').strftime('%Y-%m-%d').tolist()
print(test_dates)

test_dates[0]

['2017-12-31', '2018-03-31', '2018-06-30', '2018-09-30', '2018-12-31', '2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31', '2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31', '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31', '2022-03-31', '2022-06-30', '2022-09-30', '2022-12-31']


'2017-12-31'

In [74]:
date = datetime.datetime.strptime(test_dates[0], '%Y-%m-%d').date()
model_start_date = date 

var = 0.2
back_days = 180
front_days = 90

tickers = pd.read_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-ma' \
                              'rket-analysis/tickers.csv')

tickers = tickers.rename(columns = {"A": "Ticker"})

tickers['PV'] = tickers['Ticker'].apply(lambda x: min_var_filter(x, var=var, period=back_days))
tickers_df = pd.concat([tickers[['Ticker']], tickers['PV'].apply(pd.Series)], axis=1)
tickers_df_filtered = tickers_df[tickers_df["PV"].between(left = 0.03, right = 0.05)]

best_stocks = tickers_df_filtered["Ticker"]

start_date = model_start_date - datetime.timedelta(back_days) 
end_date = model_start_date

stock_dict = back_calculate_returns(best_stocks, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAC: Data doesn't exist for startDate = 1599019200, endDate = 1614574800
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

...

/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ZT: Data doesn't exist for startDate = 1599019200, endDate = 1614574800


/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The profit for AAP is 36.25 at buy price: 78.81 and sell price: 115.06
That equates to a 45.996700926278386% return
[*********************100%***********************]  1 of 1 completed
The profit for ABC is 23.73 at buy price: 71.9 and sell price: 95.63
That equates to a 33.00417246175243% return
[*********************100%***********************]  1 of 1 completed
The profit for AC is 6.73 at buy price: 34.28 and sell price: 35.24
That equates to a 19.632438739789965% return
[*********************100%***********************]  1 of 1 completed
The profit for ADX is 1.57 at buy price: 14.26 and sell price: 15.83
That equates to a 11.009817671809257% return
[*********************100%***********************]  1 of 1 co

ValueError: arange: cannot compute length

In [88]:
full_test = cumulative_returns_calc(test_dates, var=0.2, back_days=180, front_days=90)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAC: Data doesn't exist for startDate = 1599019200, endDate = 1614574800
[*********************100%***********************]  1 of 1 completed


/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ZT: Data doesn't exist for startDate = 1599019200, endDate = 1614574800
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The profit for AAP is 37.35 at buy price: 88.26 and sell price: 100.71
That equates to a 42.31815091774303% return
[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: 

In [9]:
def download_data(ticker, start_date=datetime.date(2018,1,1), end_date=datetime.date(2023,4,1)):
    stock_data = yf.download(ticker, start = start_date, end = end_date, interval = "1d")
    closing_price_data = np.array(stock_data["Close"])
    dates = np.array(stock_data.index)
    data = {date: [price] for date, price in zip(dates, closing_price_data)}
    df = pd.DataFrame(data=data, index=[ticker])
    return df

start_date = datetime.date(2018,1,1)
end_date = datetime.date(2023,4,1)

tickers = pd.read_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-market-analysis/tickers.csv')

tickers = tickers.rename(columns = {"A": "Ticker"}) 

tickers = tickers[0:5]
    
tickers['Prices'] = tickers['Ticker'].apply(download_data)

print(tickers)

tickers_df = pd.concat([tickers[['Ticker']], tickers['Prices'].apply(pd.Series)], axis=1)

print(tickers_df)

tickers_df.to_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-market-analysis/price_data.csv', index=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
  Ticker                                             Prices
0     AA      2018-01-02  2018-01-03  2018-01-04  2018-0...
1    AAC       2021-03-25  2021-03-26  2021-03-29  2021-...
2    AAN       2020-11-25  2020-11-27  2020-11-30  2020-...
3    AAP       2018-01-02  2018-01-03  2018-01-04  2018-...
4    AAT       2018-01-02  2018-01-03  2018-01-04  2018-...


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [62]:
start_date = datetime.date(2018,1,1)
end_date = datetime.date(2023,4,1)

old_data = yf.download("AAPL", start = start_date, end = end_date, interval = "1d")["Close"]
new_data = yf.download("MSFT", start = start_date, end = end_date, interval = "1d")["Close"]
df = pd.concat([old_data, new_data], axis=1)
df.columns = ["AAPL", "MSFT"]
print(df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                  AAPL        MSFT
Date                              
2018-01-02   43.064999   85.949997
2018-01-03   43.057499   86.349998
2018-01-04   43.257500   87.110001
2018-01-05   43.750000   88.190002
2018-01-08   43.587502   88.279999
...                ...         ...
2023-03-27  158.279999  276.380005
2023-03-28  157.649994  275.230011
2023-03-29  160.770004  280.510010
2023-03-30  162.360001  284.049988
2023-03-31  164.899994  288.299988

[1321 rows x 2 columns]


In [64]:
foo = yf.download("COIN", start = start_date, end = end_date, interval = "1d")["Close"]

df = pd.concat([df, foo], axis=1)
df.reset_index()
df

[*********************100%***********************]  1 of 1 completed


,AAPL,MSFT,Close,Close
Date,,,,
2018-01-02,43.064999,85.949997,NaN,NaN
2018-01-03,43.057499,86.349998,NaN,NaN
2018-01-04,43.257500,87.110001,NaN,NaN
2018-01-05,43.750000,88.190002,NaN,NaN
2018-01-08,43.587502,88.279999,NaN,NaN
...,...,...,...,...
2023-03-27,158.279999,276.380005,62.540001,62.540001
2023-03-28,157.649994,275.230011,63.040001,63.040001
2023-03-29,160.770004,280.510010,65.300003,65.300003


In [65]:
start_date = datetime.date(2018,1,1)
end_date = datetime.date(2023,4,1)

tickers = pd.read_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-market-analysis/tickers.csv')
tickers = tickers.rename(columns = {"A": "Ticker"})
tickers = tickers["Ticker"]

col_titles = []

df = pd.DataFrame()

for ticker in tickers:
    col_titles.append(ticker)
    data = yf.download(ticker, start = start_date, end = end_date, interval = "1d")["Close"]
    df = pd.concat([df, data], axis=1)
    df.columns = col_titles

df.to_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-market-analysis/price_data.csv', index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [70]:
all_data = pd.read_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-ma' \
                      'rket-analysis/price_data.csv')

all_data = all_data.set_index("Unnamed: 0")

all_data = all_data.rename_axis("Date")

all_data

,AA,AAC,AAN,AAP,AAT,AB,ABB,ABBV,ABC,ABEV,...,ZING,ZION,ZLAB,ZM,ZNTL,ZS,ZT,ZUMZ,ZYME,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02 00:00:00,55.169998,NaN,NaN,106.089996,38.160000,25.200001,27.020000,98.410004,94.040001,6.61,...,NaN,50.700001,22.260000,NaN,NaN,NaN,NaN,20.000000,7.933,3.009091
2018-01-03 00:00:00,54.500000,NaN,NaN,107.050003,37.820000,25.450001,27.219999,99.949997,94.389999,6.65,...,NaN,50.639999,22.350000,NaN,NaN,NaN,NaN,20.500000,7.965,3.000000
2018-01-04 00:00:00,54.700001,NaN,NaN,111.000000,37.119999,25.850000,27.450001,99.379997,94.180000,6.64,...,NaN,50.849998,22.100000,NaN,NaN,NaN,NaN,22.950001,8.430,2.954545
2018-01-05 00:00:00,54.090000,NaN,NaN,112.180000,37.139999,26.200001,27.629999,101.110001,95.320000,6.67,...,NaN,50.869999,22.809999,NaN,NaN,NaN,NaN,22.549999,8.400,2.863636
2018-01-08 00:00:00,55.000000,NaN,NaN,111.389999,37.220001,26.700001,27.600000,99.489998,96.900002,6.62,...,NaN,50.619999,22.980000,NaN,NaN,NaN,NaN,23.000000,8.310,2.909091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27 00:00:00,40.119999,10.325,9.26,114.739998,17.270000,34.860001,32.139999,157.539993,156.690002,2.66,...,10.30,29.330000,32.639999,68.570000,17.660000,111.870003,10.120,17.209999,8.100,11.380000
2023-03-28 00:00:00,40.529999,10.350,9.22,115.739998,17.540001,35.430000,32.340000,158.250000,158.000000,2.78,...,10.28,29.950001,33.840000,68.540001,16.920000,110.250000,10.175,17.389999,8.070,11.710000
2023-03-29 00:00:00,40.939999,10.330,9.03,117.550003,18.000000,35.689999,32.799999,158.000000,158.970001,2.86,...,10.27,31.040001,35.040001,70.599998,17.600000,110.699997,10.130,17.730000,8.640,11.750000


Variables we can change to tinker with the model
1. length of back/front-tests
2. PV range we're filtering between (0.03-0.05 currently)
3. stop loss

use the last 2yrs with a 1d interval to filter stocks, find stop-loss
- try a 59day with short interval to find optimal buy/sell 
- also can try 1d interval to find optimal buy/sell, but less optimistic on this

### look for medium short term vol, low long term vol

Will need to be able to download stock data. 
- Limited by data availability. Ideally would be using intraday prices going back 2yrs but beyond 60-days that data isn't available from yfinance. 
Chosen to get recent and longer term price variability (PV) separately. Looking for low on both

Retreived ticker names from https://www.nasdaq.com/market-activity/stocks/screener

Filtered out stocks smaller than 300m mktcap, manually combined nasdaq and nyse tickers (total 3800)

Can add Canadian stocks later if desired.

In [21]:
#only check short if long matches
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = model_start_date - datetime.timedelta(730) 
    twomo_start_date = model_start_date - datetime.timedelta(59) 
    end_date = model_start_date
    
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    #only check short data if long data fits, save runtime
    # below 0.03, no volume
    # above 0.055, price is too inconsistent 
    if ((0.03 < twoyr_vol) & (0.055 > twoyr_vol)): 
        short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
        twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
        twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
        twomo_avg = twomo_mid_data.mean() #calculate average 
        twomo_std_dev = twomo_mid_data.std() #calculate deviation
        twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    else:
        twomo_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [22]:
tickers = pd.read_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-ma' \
                      'rket-analysis/tickers.csv')

tickers = tickers.rename(columns = {"A": "Ticker"})

model_start_date = datetime.date(2023,3,1) #march 1, 2023

In [16]:
tickers_0_1 = tickers[0:1000] #breaking it up into segments 

tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)

tickers_0_1_df = pd.concat([tickers_0_1[['Ticker']], tickers_0_1['PV'].apply(pd.Series)], axis=1)

print(tickers_0_1_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

KeyboardInterrupt: 

In [ ]:
tickers_1_2 = tickers[1000:2000] 

tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)

tickers_1_2_df = pd.concat([tickers_1_2[['Ticker']], tickers_1_2['PV'].apply(pd.Series)], axis=1)

print(tickers_1_2_df)

In [ ]:
tickers_2_3 = tickers[2000:3000] 

tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)

tickers_2_3_df = pd.concat([tickers_2_3[['Ticker']], tickers_2_3['PV'].apply(pd.Series)], axis=1)

print(tickers_2_3_df)

In [ ]:
tickers_3_end = tickers[3000:] 

tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)

tickers_3_end_df = pd.concat([tickers_3_end[['Ticker']], tickers_3_end['PV'].apply(pd.Series)], axis=1)

print(tickers_3_end_df)

In [ ]:
#opting to set min variability instead of volume to filter based on liquidity but could change later 
tickers_0_1_df_filtered = tickers_0_1_df[tickers_0_1_df["Short PV"].between(left = 0, right = 1)]
tickers_1_2_df_filtered = tickers_1_2_df[tickers_1_2_df["Short PV"].between(left = 0, right = 1)]
tickers_2_3_df_filtered = tickers_2_3_df[tickers_2_3_df["Short PV"].between(left = 0, right = 1)]
tickers_3_end_df_filtered = tickers_3_end_df[tickers_3_end_df["Short PV"].between(left = 0, right = 1)]

print(tickers_0_1_df_filtered)
print(tickers_1_2_df_filtered)
print(tickers_2_3_df_filtered)
print(tickers_3_end_df_filtered)

In [8]:
best_stocks = ["GUD.TO", "HONE", "CBSH", "FNLC", "AMCR", "BDX", "CAPL", "CHE", "CP", "CTBI", "DCOM", "FRPH", "GSBC", "IEP", 
               "INDB", "TOWN", "WABC"]

start_date = datetime.date(2023,3,1) - datetime.timedelta(730) 
end_date = datetime.date(2023,3,1)

In [15]:
def calculate_returns(list_of_stocks, start_date=start_date, end_date=end_date):
    """
    Print the optimal buy and sell price of each stock in the given list over the given range
    Also gives total % returns over that period
    """
    
    # need to add these globals since vectorizing, otherwise the program complains that it's calling  
    #.  variables before they're defined
    global owned
    global max_profit
    global best_sell
    global best_buy

    for i, stock in enumerate(list_of_stocks):
        info = yf.download(stock, start=start_date, end=end_date)[["High", "Low"]]

        info = info.round(2)

        owned = False
        max_profit = 0.
        best_buy = 0.
        best_sell = 0.

        min_low = np.min(info["Low"])
        max_high = np.max(info["High"])

        #incrementing at 1% of buy price to reduce runtime
        #ideally would do 0.01 for all but that's probably more trouble than it's worth at high prices
        increment = min_low/100

        buy_prices = np.arange(min_low, max_high, increment)

        def buy(buy_price):

            global owned
            global max_profit
            global best_sell
            global best_buy

            sell_prices = np.arange(buy_price, max_high, increment)

            def sell(sell_price):
                global owned
                global max_profit
                global best_sell
                global best_buy

                profit = 0.

                for index, row in info.iterrows():
                    if (owned):
                        if (row["High"] >= sell_price):
                            profit += (sell_price - buy_price)
                            owned = False
                    else:
                        if (row["Low"] <= buy_price):
                            owned = True

                    if (profit > max_profit):
                        max_profit = round(profit, 2)
                        best_sell = round(sell_price, 2)
                        best_buy = round(buy_price, 2)

            sell_vec = np.vectorize(sell)
            sell_vec(sell_prices)

        buy_vec = np.vectorize(buy)
        buy_vec(buy_prices)

        percent_return = max_profit / best_buy * 100 

        print(f"The profit for {stock} is {max_profit} at buy price: {best_buy} and sell price: {best_sell}")
        print(f"That equates to a {percent_return}% return")


In [16]:
calculate_returns(best_stocks)

[*********************100%***********************]  1 of 1 completed
The profit for GUD.TO is 3.66 at buy price: 5.22 and sell price: 5.47
That equates to a 70.11494252873564% return
[*********************100%***********************]  1 of 1 completed
The profit for HONE is 10.87 at buy price: 13.53 and sell price: 14.13
That equates to a 80.33998521803399% return
[*********************100%***********************]  1 of 1 completed
The profit for CBSH is 46.36 at buy price: 67.99 and sell price: 71.08
That equates to a 68.18649801441389% return
[*********************100%***********************]  1 of 1 completed
The profit for FNLC is 23.54 at buy price: 28.66 and sell price: 29.68
That equates to a 82.13538032100487% return
[*********************100%***********************]  1 of 1 completed
The profit for AMCR is 6.2 at buy price: 11.16 and sell price: 12.71
That equates to a 55.55555555555556% return
[*********************100%***********************]  1 of 1 completed
The profit for

Next steps: decide on stop loss
- drops below min_low
    - we have defined the range in which we think we can predict prices, and if it 
      leaves that range we discard
- Could do some optimization stuff here depending on how testing goes. If there's significant improvement in returns then could do something like a sell if the price drops X% below the buy price, and a permanent exit if it drops below min_low


TESTING: 

In creating this model I used the present day as the starting point. This was an intentional decision to have a quick check of my hypothesis (ie. what if GUD.TO was a unicorn and there were no other stocks with the chart patterns I'm looking for today, why bother creating a model that worked years ago?). Starting today (March 2023) also has the distinct advantage of catching a diversity of environments in the backtest (strong 2021, weak 2022). 

The problem is that these best tickers provide training data, but no test set until enough time passes.

To test the model, need to replicate the steps above using different start dates and calculate returns in the following (year?). 

### Testing - start date 2022,3,1

In [45]:
# have to change the function slightly since there's no 30m data for times older than 2mo. 
# might involve slightly more charts to look over if the filtering is worse
def old_data_short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 1DAY interval. Includes high and low price from each period. 
       Have to do 1d interval since the data is old.
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = model_start_date - datetime.timedelta(730) 
    twomo_start_date = model_start_date - datetime.timedelta(59) 
    end_date = model_start_date
    
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    #only check short data if long data fits, save runtime
    # below 0.03, no volume
    # above 0.055, price is too inconsistent 
    if ((0.03 < twoyr_vol) & (0.055 > twoyr_vol)): 
        short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "1d")[["High", "Low"]]   
        twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
        twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
        twomo_avg = twomo_mid_data.mean() #calculate average 
        twomo_std_dev = twomo_mid_data.std() #calculate deviation
        twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    else:
        twomo_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [46]:
#using present day tickers may bias the model as companies that have since been delisted aren't included. 
# assuming that number is small, and the effect is negligible, but it's something that could be addressed later 
# this does, however, limit how far back I can test without updating the ticker list 
# even starting in 2022 already a lot of stock data not found (newer IPOs)
tickers = pd.read_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-ma' \
                      'rket-analysis/tickers.csv')

tickers = tickers.rename(columns = {"A": "Ticker"})

#what if we start in 2022?
model_start_date = datetime.date(2022,3,1) 

In [47]:
tickers_0_1 = tickers[0:1000] #breaking it up into segments 

tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(old_data_short_long_price_variability)

tickers_0_1_df = pd.concat([tickers_0_1[['Ticker']], tickers_0_1['PV'].apply(pd.Series)], axis=1)

print(tickers_0_1_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- HLN: Data doesn't exist for startDate = 1583038800, endDate = 1646110800
[*********************100%***********************]  1 of 1 completed


/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/433334890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(old_data_short_long_price_variability)


    Ticker  Short PV   Long PV
0       AA      -1.0  0.601502
1      AAC      -1.0  0.004809
2      AAN      -1.0  0.188377
3      AAP      -1.0  0.218387
4      AAT      -1.0  0.169416
..     ...       ...       ...
995    IQV      -1.0  0.245738
996     IR      -1.0  0.243617
997    IRM      -1.0  0.243190
998    IRS      -1.0  0.137326
999    IRT      -1.0  0.312899

[1000 rows x 3 columns]


In [48]:
tickers_1_2 = tickers[1000:2000] 

tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(old_data_short_long_price_variability)

tickers_1_2_df = pd.concat([tickers_1_2[['Ticker']], tickers_1_2['PV'].apply(pd.Series)], axis=1)

print(tickers_1_2_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/1572123670.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(old_data_short_long_price_variability)


     Ticker  Short PV   Long PV
1000    ISD      -1.0  0.096124
1001     IT      -1.0  0.391473
1002   ITCB      -1.0  0.191112
1003   ITGR      -1.0  0.142078
1004    ITT      -1.0  0.227281
...     ...       ...       ...
1995    WPP      -1.0  0.263236
1996    WRB      -1.0  0.145235
1997   WRBY      -1.0  0.213022
1998    WRK      -1.0  0.229608
1999    WSM      -1.0  0.353708

[1000 rows x 3 columns]


In [49]:
tickers_2_3 = tickers[2000:3000] 

tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(old_data_short_long_price_variability)

tickers_2_3_df = pd.concat([tickers_2_3[['Ticker']], tickers_2_3['PV'].apply(pd.Series)], axis=1)

print(tickers_2_3_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ACDC: Data doesn't exist for startDate = 1583038800, endDate = 1646110800
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- JGGC: Data doesn't exist for startDate = 1583038800, endDate = 1646110800
[*********************100%***********************]  1 of 1 completed


/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/4262042292.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(old_data_short_long_price_variability)


     Ticker  Short PV   Long PV
2000    WSO      -1.0  0.187960
2001    WSR      -1.0  0.195632
2002    WST      -1.0  0.277002
2003     WT      -1.0  0.272891
2004    WTI      -1.0  0.305366
...     ...       ...       ...
2995   LULU      -1.0  0.188926
2996   LUNA      -1.0  0.250103
2997   LUNG      -1.0  0.226642
2998   LUNR      -1.0  0.002952
2999   LVRA      -1.0  0.006719

[1000 rows x 3 columns]


In [50]:
tickers_3_end = tickers[3000:] 

tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(old_data_short_long_price_variability)

tickers_3_end_df = pd.concat([tickers_3_end[['Ticker']], tickers_3_end['PV'].apply(pd.Series)], axis=1)

print(tickers_3_end_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/3431387241.py:19: RuntimeWarning: Mean of empty slice.
  twoyr_avg = twoyr_mid_data.mean()
/Users/kaigroden-gilchrist/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
     Ticker  Short PV   Long PV
3000   LWLG      -1.0  1.127261
3001     LX      -1.0  0.317714
3002   LXRX      -1.0  0.510970
3003   LYEL      -1.0  0.329561
3004   LYFT      -1.0  0.279552
...     ...       ...       ...
3793     ZS      -1.0  0.411389
3794     ZT      -1.0  0.004985
3795   ZUMZ      -1.0  0.264370
3796   ZYME      -1.0  0.320924
3797   ZYXI      -1.0  0.253034

[798 rows x 3 columns]


/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/858962260.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(old_data_short_long_price_variability)


In [51]:
#opting to set min variability instead of volume to filter based on liquidity but could change later 
tickers_0_1_df_filtered = tickers_0_1_df[tickers_0_1_df["Short PV"].between(left = 0, right = 1)]
tickers_1_2_df_filtered = tickers_1_2_df[tickers_1_2_df["Short PV"].between(left = 0, right = 1)]
tickers_2_3_df_filtered = tickers_2_3_df[tickers_2_3_df["Short PV"].between(left = 0, right = 1)]
tickers_3_end_df_filtered = tickers_3_end_df[tickers_3_end_df["Short PV"].between(left = 0, right = 1)]

print(tickers_0_1_df_filtered)
print(tickers_1_2_df_filtered)
print(tickers_2_3_df_filtered)
print(tickers_3_end_df_filtered)

    Ticker  Short PV   Long PV
156    ATO  0.011878  0.053551
199    BAX  0.017666  0.048570
207    BBN  0.044544  0.051204
226    BDX  0.027978  0.047585
323    BTT  0.017864  0.046139
365    CBL  0.035754  0.049094
398    CHT  0.019576  0.053439
428    CMS  0.015845  0.048291
429   CMTG  0.028840  0.035309
515      D  0.010985  0.046600
560    DLY  0.028687  0.045971
566    DNP  0.007956  0.042379
665     ES  0.033697  0.047996
732   FINS  0.012978  0.044856
791   GBAB  0.048572  0.052658
869    GUG  0.037920  0.042441
     Ticker  Short PV   Long PV
1040      K  0.024151  0.043568
1059    KMB  0.037135  0.051304
1082    KTF  0.034659  0.054921
1135   LVWR  0.003853  0.034154
1175   MEGI  0.028832  0.033476
1186    MHN  0.032859  0.053362
1188    MIN  0.033719  0.034594
1198    MMD  0.026777  0.048884
1203    MMU  0.031121  0.054122
1220    MRK  0.027802  0.045830
1261    NAC  0.041745  0.049240
1263    NAN  0.037876  0.053353
1273   NDMO  0.054393  0.051872
1275    NEA  0.037820  0.

check each stock chart for the 2yrs before the start date, pick out good ones.

ATO - not bad but breaks out at the end
BAX - falls off a cliff right at the end
BBN - breaks down at the end
**BTT - starts low (pandemic) but turns very good**
CBL - no
CHT - no, growing
**CMS - quite good**
CMTG - no, new listing
**D - yes**
DLY - no, bigger swings than wanted
**DNP - yes**
**ES - YES**
FINS - decent for periods but overall not strong enough
GBAB - no, too big of swings
GUG - no
**K - YES**
**KMB - yes**
KTF - not awful but not choppy enough
LVWR - not enough info
MEGI - no
MHN - not awful but not choppy enough
MIN - good but sinking towards the end
MMD - not a flat chop
MMU - not a flat chop
**MRK - YES**
NAC - not a flat chop
NAN - not a flat chop
NDMO - not a flat chop
NEA - not a flat chop
NKX - not a flat chop
NMAI - no
NPFD - no
NQP - not a flat chop
NRK - not a flat chop
**NVS - YES**
ONL - no
PDO - not a flat chop
PPT - falling
PRS - not a flat chop
PTA - falling
RFMZ - no
SDHY - falling
TCN - no
WDI - no
**WEC - yes**
ACT - no
**AEP - yes**
BCSA - no
BLFY - no
**DGICA - yes**
MLTX - last 1yr looks great, volatile beyond
PROC - no, falling
**SNY - yes**
TBLD - no
TPG - no
**WABC - pretty good**


different bins of stocks - at a glance, most fit into one of these 
- a LOT of municipal bond funds, dont look great for my purposes, rising from pandemic low then fall in March 2022 as rates rise
- blue chips look good
- energy looks good


In [104]:
def back_calculate_returns(list_of_stocks, start_date=start_date, end_date=end_date):
    """
    Print the optimal buy and sell price of each stock in the given list over the given range
    Also gives total % returns over that period
    """
    
    returns = {}
    
    # need to add these globals since vectorizing, otherwise the program complains that it's calling  
    #.  variables before they're defined
    global owned
    global max_profit
    global best_sell
    global best_buy

    for i, stock in enumerate(list_of_stocks):
        info = yf.download(stock, start=start_date, end=end_date)[["High", "Low"]]

        info = info.round(2)

        owned = False
        max_profit = 0.
        best_buy = 0.
        best_sell = 0.

        min_low = np.min(info["Low"])
        max_high = np.max(info["High"])

        #incrementing at 1% of buy price to reduce runtime
        #ideally would do 0.01 for all but that's probably more trouble than it's worth at high prices
        increment = min_low/100

        buy_prices = np.arange(min_low, max_high, increment)

        def buy(buy_price):

            global owned
            global max_profit
            global best_sell
            global best_buy

            sell_prices = np.arange(buy_price, max_high, increment)

            def sell(sell_price):
                global owned
                global max_profit
                global best_sell
                global best_buy

                profit = 0.

                for index, row in info.iterrows():
                    if (owned):
                        if (row["High"] >= sell_price):
                            profit += (sell_price - buy_price)
                            owned = False
                    else:
                        if (row["Low"] <= buy_price):
                            owned = True

                    if (profit > max_profit):
                        max_profit = round(profit, 2)
                        best_sell = round(sell_price, 2)
                        best_buy = round(buy_price, 2)

            sell_vec = np.vectorize(sell)
            sell_vec(sell_prices)

        buy_vec = np.vectorize(buy)
        buy_vec(buy_prices)

        percent_return = max_profit / best_buy * 100 

        print(f"The profit for {stock} is {max_profit} at buy price: {best_buy} and sell price: {best_sell}")
        print(f"That equates to a {percent_return}% return")
        
        returns.update({stock : {"buy_price" : best_buy, "sell_price" : best_sell, 
            "min_low" : min_low, "max_high" : max_high}})
        
    return returns


In [31]:
best_stocks_20200301_20220301 = ["BTT", "CMS", "D", "DNP", "ES", "K", "KMB", "MRK", "NVS", "WEC", "AEP", "DGICA",
                                "SNY", "WABC"]

model_start_date = datetime.date(2022,3,1) 

start_date = model_start_date - datetime.timedelta(730) 
end_date = model_start_date

backreturns_20200301_20220301 = back_calculate_returns(best_stocks_20200301_20220301, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for BTT is 8.38 at buy price: 18.62 and sell price: 27.0
That equates to a 45.005370569280345% return
[*********************100%***********************]  1 of 1 completed
The profit for CMS is 35.44 at buy price: 58.92 and sell price: 63.98
That equates to a 60.14935505770536% return
[*********************100%***********************]  1 of 1 completed
The profit for D is 46.23 at buy price: 71.66 and sell price: 80.91
That equates to a 64.51297795143735% return
[*********************100%***********************]  1 of 1 completed
The profit for DNP is 5.81 at buy price: 6.76 and sell price: 12.57
That equates to a 85.94674556213018% return
[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: 

Some of these are not working as intended. The recommendation is to catch the covid low in March 2020 and ride it to the peak, instead of making successive buys and sells. 

For now, try only calculating back 1yr to avoid that covid low. 

Alternatively consider eliminate stocks which dipped too significantly during March 2020 (may be ideal).

In [32]:
best_stocks_20200301_20220301 = ["BTT", "CMS", "D", "DNP", "ES", "K", "KMB", "MRK", "NVS", "WEC", "AEP", "DGICA",
                                "SNY", "WABC"]

model_start_date = datetime.date(2022,3,1) 

start_date = model_start_date - datetime.timedelta(365) #1yr back now
end_date = model_start_date

backreturns_20210301_20220301 = back_calculate_returns(best_stocks_20200301_20220301, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for BTT is 2.89 at buy price: 24.1 and sell price: 26.99
That equates to a 11.991701244813278% return
[*********************100%***********************]  1 of 1 completed
The profit for CMS is 19.15 at buy price: 59.04 and sell price: 65.42
That equates to a 32.43563685636856% return
[*********************100%***********************]  1 of 1 completed
The profit for D is 20.35 at buy price: 73.28 and sell price: 80.06
That equates to a 27.77019650655022% return
[*********************100%***********************]  1 of 1 completed
The profit for DNP is 1.46 at buy price: 9.75 and sell price: 11.21
That equates to a 14.974358974358973% return
[*********************100%***********************]  1 of 1 completed
The profit for ES is 29.89 at buy price: 80.47 and sell price: 90.44
That equates to a 37.14427737044861% return
[*********************100%***********************]  1 of 1 completed
The profit for K is 2

Still some with a single optimized buy/sell. Going to eliminate those and continue with the forward test on others


In [56]:
#print(backreturns_20210301_20220301)

for key in backreturns_20210301_20220301:
    prices = (backreturns_20210301_20220301[key])
    buy_price = prices["buy_price"]
    sell_price = prices["sell_price"]
    print(buy_price, sell_price)
    

24.1 26.99
59.04 65.42
73.28 80.06
9.75 11.21
80.47 90.44
62.43 64.13
130.28 137.8
68.44 91.03
82.51 90.45
94.45 96.88
82.46 89.95
13.15 16.44
47.43 52.4
56.43 57.5


In [59]:
def front_calculate_returns(dict_of_stocks, start_date, end_date):
    """
    Front calculate returns using given dict with key: ticker vals: buy/sell prices, backwards looking 
       max high/min low 
    Sell out if price goes above max high/min low
    """
    
    global owned

    for ticker in dict_of_stocks:
        
        prices = dict_of_stocks[ticker]
        buy_price = prices["buy_price"]
        sell_price = prices["sell_price"]
        min_low = prices["min_low"]
        max_high = prices["max_high"]
        
        info = yf.download(ticker, start=start_date, end=end_date)[["High", "Low"]]

        info = info.round(2)

        owned = False
        
        profit = 0.

        for index, row in info.iterrows():
            #break if you leave the bounds of the training data
            if (row["Low"] < min_low):
                if (owned):
                    profit += row["Low"] - buy_price
                break
            if (row["High"] > max_high):
                if (owned):
                    profit += row["High"] - buy_price
                break
                
            if (owned):
                if (row["High"] >= sell_price):
                    profit += (sell_price - buy_price)
                    owned = False
            else:
                if (row["Low"] <= buy_price):
                    owned = True

        percent_return = profit / buy_price * 100 

        print(f"The profit for {ticker} is {profit}")
        print(f"That equates to a {percent_return}% return")


In [58]:
#eliminating BTT, DNP, MRK, DGICA
best_stocks_20210301_20220301 = ["CMS", "D", "ES", "K", "KMB", "NVS", "WEC", "AEP", "SNY", "WABC"]

model_start_date = datetime.date(2022,3,1) 

start_date = model_start_date - datetime.timedelta(365) #1yr back
end_date = model_start_date

stock_dict = back_calculate_returns(best_stocks_20210301_20220301, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for CMS is 19.15 at buy price: 59.04 and sell price: 65.42
That equates to a 32.43563685636856% return
[*********************100%***********************]  1 of 1 completed
The profit for D is 20.35 at buy price: 73.28 and sell price: 80.06
That equates to a 27.77019650655022% return
[*********************100%***********************]  1 of 1 completed
The profit for ES is 29.89 at buy price: 80.47 and sell price: 90.44
That equates to a 37.14427737044861% return
[*********************100%***********************]  1 of 1 completed
The profit for K is 23.84 at buy price: 62.43 and sell price: 64.13
That equates to a 38.18676918148326% return
[*********************100%***********************]  1 of 1 completed
The profit for KMB is 45.1 at buy price: 130.28 and sell price: 137.8
That equates to a 34.617746392385634% return
[*********************100%***********************]  1 of 1 completed
The profit for NVS i

In [60]:
model_start_date = datetime.date(2022,3,1) 

start_date = model_start_date 
end_date = model_start_date + datetime.timedelta(365) #1yr forward now

front_calculate_returns(stock_dict, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for CMS is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for D is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for ES is 9.969999999999999
That equates to a 12.389710451099788% return
[*********************100%***********************]  1 of 1 completed
The profit for K is 1.6999999999999957
That equates to a 2.723049815793682% return
[*********************100%***********************]  1 of 1 completed
The profit for KMB is -7.159999999999997
That equates to a -5.4958550813632145% return
[*********************100%***********************]  1 of 1 completed
The profit for NVS is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for WEC is 2.4299999999999926
That equates to a 2.5727898358919985% return
[**********

These returns look pretty bad. In too many cases, we're exiting immediately.
What if I don't exit if I break the training bounds on the upside, only downside?
- Could also add a limit on the upside beyond just the training bounds limit

In [78]:
def front_calculate_returns(dict_of_stocks, start_date, end_date):
    """
    Front calculate returns using given dict with key: ticker vals: buy/sell prices, backwards looking 
       max high/min low 
    Sell out if price goes above max high/min low
    """
    
    global owned
    
    avg_returns_list = []

    for ticker in dict_of_stocks:
        
        prices = dict_of_stocks[ticker]
        buy_price = prices["buy_price"]
        sell_price = prices["sell_price"]
        min_low = prices["min_low"]
        max_high = prices["max_high"]
        
        info = yf.download(ticker, start=start_date, end=end_date)[["High", "Low"]]

        info = info.round(2)

        owned = False
        
        profit = 0.

        for index, row in info.iterrows():
            #break if you leave the bounds of the training data
            if (row["Low"] < min_low):
                if (owned):
                    profit += row["Low"] - buy_price
                break
            #removed exit on upside out of bounds
                
            if (owned):
                if (row["High"] >= sell_price):
                    profit += (sell_price - buy_price)
                    owned = False
            else:
                if (row["Low"] <= buy_price):
                    owned = True

        percent_return = profit / buy_price * 100

        print(f"The profit for {ticker} is {profit}")
        print(f"That equates to a {percent_return}% return")
        
        avg_returns_list.append(percent_return)
    
    equal_weight_avg_return = np.mean(avg_returns_list)
    
    print(avg_returns_list)
    print (f"The equal weight average returns are: {equal_weight_avg_return}")


In [82]:
model_start_date = datetime.date(2022,3,1) 

start_date = model_start_date 
end_date = model_start_date + datetime.timedelta(365) #1yr forward now

front_calculate_returns(stock_dict, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
The profit for CMS is -6.630000000000003
That equates to a -11.229674796747972% return
[*********************100%***********************]  1 of 1 completed
The profit for D is -0.6800000000000068
That equates to a -0.9279475982532844% return
[*********************100%***********************]  1 of 1 completed
The profit for ES is 14.86
That equates to a 18.466509258108612% return
[*********************100%***********************]  1 of 1 completed
The profit for K is 1.6999999999999957
That equates to a 2.723049815793682% return
[*********************100%***********************]  1 of 1 completed
The profit for KMB is -7.159999999999997
That equates to a -5.4958550813632145% return
[*********************100%***********************]  1 of 1 completed
The profit for NVS is 0.0
That equates to a 0.0% return
[*********************100%***********************]  1 of 1 completed
The profit for WEC is 12.149999999999963
That 

An average of 3% returns... not great. Will need to test over other time periods
- Not too surprising it's low. No replacement factored in here.
- Can also test different lengths of time to back/front test
- Idea is to update monthly, so front-test of 30day at a time then update stocks
    - That's a lot of scanning charts
        - Will need a more automated way to scan charts

In [46]:
info = yf.download("HONE", start=start_date, end=end_date)[["High", "Low"]]

min_low = info.Low.min()
max_high = info.High.max()
    
for date in info.keys:
    if (owned):
        if (info.High >= sell_price):
            profit += sell_price - buy_price
            owned = False
    else:
          if (info.Low <= buy_price):
            owned = True
    print(profit)

[*********************100%***********************]  1 of 1 completed


TypeError: 'method' object is not iterable

In [ ]:
import yfinance as yf #contains stock price data
import numpy as np
import pendulum 
import matplotlib.pyplot as plt 

stock_info = yf.Ticker('TSLA').info

market_price = stock_info['regularMarketPrice']
previous_close_price = stock_info['regularMarketPreviousClose']
print('market price ', market_price)
print('previous close price ', previous_close_price)

price_history = yf.Ticker('TSLA').history(period='2y', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                   interval='1mo', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                   actions=False)
time_series = list(price_history['Open'])
dt_list = [pendulum.parse(str(dt)).float_timestamp for dt in list(price_history.index)]
plt.style.use('dark_background')
plt.plot(dt_list, time_series, linewidth=2)

In [ ]:
def short_long_returns_volatility_ratio(short_long_list):
    
    """
    Calc the ratio between short (2mo) and long (2yr) term daily volatility of log returns.
    Higher output means high short term vol relative to long term.
    
    Literature uses log returns to calculate volatility. For my purposes I'm not sure this is necessary.
    I only care about deviation in price so a more simple approach should suffice. 
    """
    
    short_data = short_long_list[0]
    long_data = short_long_list[1]
    
    twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twomo_log_returns = twomo_midpoint.assign(log_returns = lambda x: np.log(x.Mid / x.Mid.shift()))
    twomo_std_dev = np.array(twomo_log_returns["log_returns"])[1:].std()
    twomo_vol = twomo_std_dev * (59**0.5) 
    
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2)
    twoyr_log_returns = twoyr_midpoint.assign(log_returns = lambda x: np.log(x.Mid / x.Mid.shift()))
    twoyr_std_dev = np.array(twoyr_log_returns["log_returns"])[1:].std()
    twoyr_vol = twoyr_std_dev * (730**0.5)
    
    return twomo_vol, twoyr_vol

In [ ]:
#only check long if short matches - not using this one
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())

    short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
    twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
    twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
    twomo_avg = twomo_mid_data.mean() #calculate average 
    twomo_std_dev = twomo_mid_data.std() #calculate deviation
    twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    
    if ((0.01 < twomo_vol) & (0.015 > twomo_vol)): #only check long data if short data fits, save runtime
        long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
        twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
        twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
        twoyr_avg = twoyr_mid_data.mean()
        twoyr_std_dev = twoyr_mid_data.std()
        twoyr_vol = twoyr_std_dev / twoyr_avg
    else:
        twoyr_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 


In [ ]:
def download_from_ticker(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())

    short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    return [short_data, long_data]


def short_long_price_variability(short_long_list):
    
    """
    Calc short (2mo) and long (2yr) term price variability (price adjusted standard deviation).
    """
    
    short_data = short_long_list[0]
    long_data = short_long_list[1]
    
    twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
    twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
    twomo_avg = twomo_mid_data.mean() #calculate average 
    twomo_std_dev = twomo_mid_data.std() #calculate deviation
    twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    return twomo_vol, twoyr_vol

PNM is flat in the 2yr, but not enough chop for this 
TXO is a recent IPO, not enough data.
BCSA is not flat, slowly rising. Also a recent (~1yr) IPO so not enough data
DCRD is a SPAC (explains long term PV). Recent PV too crazy (up 47% Feb 13-20)
HONE is great. Exactly what we're looking for (although ideally no dividend)
LANDM looks like it was stable, but now trending downwards in the last 6mo. Not ideal. 

Based on HONE's Long PV of 0.34, going to expand Long PV range to 0.42 to see if I can get more hits slightly higher. Moving min to 0.3 since I've screened below and don't see anything. 

EQC looked good, but just broke down after a special div payment
JNJ is decent. Larger swings in price but may be enough of them not to rule it out.
CBSH looks good.
CZNC declining 1yr MA not ideal. 
ENLT recent IPO. No
FNLC looks good.
MLYS recent IPO
SKWD recent IPO.

CBSH and FNLC are towards the top end of the Long PV range, may as well check higher. 

ADC looks decent. Bigger dif between highs and lows than I'd like.
AEE looks decent. Looks like a general uptrend but not too pronounced
AMCR looks nice. 
AVA is pretty good. Last 1yr better than previous.
BDJ looks to be trending down slightly. 
BDX looks nice. 
BXMX looks decent. Very cyclical around the dividend. Maybe slightly trending down. 
CAG is ok. Bigger dif between highs and lows than I'd like.
CAJ was looking good. Fell off in last 6mo. 
CAPL looks nice. Farthest end of 2yr is uptrend but since then nice. 
CHE looks good. 
CL is ok. Bigger dif between highs and lows than I'd like.
CNA not great. Wide wavelength. 
CP looks great. 
CRBG is recent IPO. No. 
DIAX looks general downtrend. No.
DNP no.
DPG looks pretty good. 
DUK not great. 
EIG looks ok. Something strange big movement in Oct 2022
ES was good. May have broken down recently. 
FCPT is ok. Bigger dif between highs and lows than I'd like
FEI is good. Far end of 2yr rising but flat since. 
GHC is ok. Not great. 
GIB no.
HE pretty good. 
HFRO no. 
HMN slight downtrend. Potentially ok.
HRL no. Strong support but hitting lower highs. 
HTD not great. 
IBM is ok. Bigger dif between highs and lows than I'd like.
IDA is ok. Bigger dif between highs and lows than I'd like.
IGD no. 
INGR not great. 
KMB is ok. Bigger dif between highs and lows than I'd like.
KMI is ok. Bigger dif between highs and lows than I'd like.
LADR is ok. Don't like the big drop in Oct 22.
MMP in a slight uptrend but still looks good. 
MSM is ok. Bigger dif between highs and lows than I'd like.
NID no. 
NNN is ok. Bigger dif between highs and lows than I'd like.
NVS is ok. Bigger dif between highs and lows than I'd like.
O is ok. Don't like the big drop in Oct 22.
OFC is ok. Don't like the big drop in Oct 22.
PB is pretty good. 
PFS is ok. Bigger dif between highs and lows than I'd like.
PM is ok. Bigger dif between highs and lows than I'd like.
POR no. 
PPL looks ok, but don't like that it's hitting lower lows since Nov 21. 
PRS no. 
SCM just broke out. No. 
SON looks pretty good. 
SPH looks pretty good. 
STZ is ok. Bigger dif between highs and lows than I'd like.
SYY is pretty good. Bigger dif between highs and lows than I'd like.
THG no. 
VOYA just broke out. No. 
WEC is ok. Bigger dif between highs and lows than I'd like.
WMT is ok. Bigger dif between highs and lows than I'd like.
WPC no. 
WTRG is ok. Bigger dif between highs and lows than I'd like.
ARCC no. 
AROW no. 
BTWN no volume. 
BWB is ok but looks too long wavelength. 
CAC no. In slight downtrend. 
CATC looks pretty good. 
CATY looks ok. 
CCNE looks pretty good. 
CGBD was flat. Breaking out now. 
CIVB no. Trending down. 
CTBI looks good. 
DCOM looks good. 
DGICA is ok. Bigger dif between highs and lows than I'd like.
EVRG is ok. Bigger dif between highs and lows than I'd like.
FRPH looked a little off far end of 2yr. Very good now. 
GGAA no. SPAC.
GLPI slight uptrend. No. 
GNTY no. 
GSBC good in the last year. 
IEP good for the past year. 
INDB pretty good for the past year. 
KDP decent. 
MDLZ decent. High amplitude. 
MGEE no. 
MOFG pretty good. 
MSBI pretty good. 
NMFC no. Was good but now too big amp. 
NWBI no. Way too big amp. 
OBIO no. Spac?
OBNK decent. Maybe slight downtrend. 
OCSL no. 
REYN big amplitude. Prob not. 
RPRX big aplitude. Prob not. 
SAFT no. Slight uptrend, large amp. 
SIRI would have been great but just crashed. 
TCPC decent. Slight downtrend. 
THFF is ok. 
TOWN is great. 
VGAS no. SAPC. 
VLGEA is decent. Don't like the big drop in Oct 22.
WABC good in the past 1.5yr.
WAFD is ok. 
WALD no. SPAC.
XEL not great. High amplitude. 

Looks like the upper end of Long PV from this batch are generally not great (with a couple exceptions), meaning we've probably found our upper limit. 

The ones that look really good from this are:
      Short     Long
HONE  0.015847  0.034489
CBSH  0.018631  0.041063
FNLC  0.015954  0.040111
AMCR  0.029136  0.043482
BDX   0.023021  0.043537
CAPL  0.036246  0.056862 
CHE   0.016904  0.046703
CP    0.019901  0.053180
CTBI  0.038825  0.048464
DCOM  0.030092  0.059724
FRPH  0.023382  0.050308
GSBC  0.016853  0.046868
IEP   0.017944  0.054363
INDB  0.023662  0.059701
TOWN  0.016220  0.050351
WABC  0.019999  0.048959

Next model prob want 0.3 < Long PV < 0.55 & 

Can use optimization to figure out buys (at least initially?)

In [14]:
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = model_start_date - datetime.timedelta(730) 
    twomo_start_date = model_start_date - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())
    
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    if ((0.025 < twoyr_vol) & (0.035 > twoyr_vol)): #only check long data if long data fits, save runtime
        short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
        twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
        twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
        twomo_avg = twomo_mid_data.mean() #calculate average 
        twomo_std_dev = twomo_mid_data.std() #calculate deviation
        twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    else:
        twomo_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [11]:
# testing on our poster-child, expecting low price variability
print(short_long_price_variability("GUD.TO"))

# expecting pretty high volatility on this one
print(short_long_price_variability("BBBY"))

#works on indices as well
print(short_long_price_variability("^DJI"))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
{'Short PV': 0.01343381341107851, 'Long PV': 0.030393921179468653}
[*********************100%***********************]  1 of 1 completed
{'Short PV': -1, 'Long PV': 0.6331323763658447}
[*********************100%***********************]  1 of 1 completed
{'Short PV': -1, 'Long PV': 0.04753292545283113}


It looks like BBBY has about 25x short and 20x long PV compared to GUD.TO. So the function is doing what it's supposed to.

DJI has comparable short term PV, and about 1.5x long.

GUD.TO is remarkably boring. 

In [18]:
tickers_0_1 = tickers[0:100] #breaking it up into segments 

tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)

tickers_0_1_df = pd.concat([tickers_0_1[['Ticker']], tickers_0_1['PV'].apply(pd.Series)], axis=1)

print(tickers_0_1_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_93128/969744139.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)


In [ ]:
tickers_1_2 = tickers[1000:2000] 

tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)

tickers_1_2_df = pd.concat([tickers_1_2[['Ticker']], tickers_1_2['PV'].apply(pd.Series)], axis=1)

print(tickers_1_2_df)

In [ ]:
tickers_2_3 = tickers[2000:3000] 

tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)

tickers_2_3_df = pd.concat([tickers_2_3[['Ticker']], tickers_2_3['PV'].apply(pd.Series)], axis=1)

print(tickers_2_3_df)

In [ ]:
tickers_3_end = tickers[3000:] 

tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)

tickers_3_end_df = pd.concat([tickers_3_end[['Ticker']], tickers_3_end['PV'].apply(pd.Series)], axis=1)

print(tickers_3_end_df)

In [19]:
tickers_0_1_df_filtered = tickers_0_1_df[tickers_0_1_df["Short PV"].between(left = 0, right = 1)]
tickers_1_2_df_filtered = tickers_1_2_df[tickers_1_2_df["Short PV"].between(left = 0, right = 1)]
tickers_2_3_df_filtered = tickers_2_3_df[tickers_2_3_df["Short PV"].between(left = 0, right = 1)]
tickers_3_end_df_filtered = tickers_3_end_df[tickers_3_end_df["Short PV"].between(left = 0, right = 1)]

print(tickers_0_1_df_filtered)
print(tickers_1_2_df_filtered)
print(tickers_2_3_df_filtered)
print(tickers_3_end_df_filtered)

NameError: name 'tickers_1_2_df' is not defined

In [ ]:
#only check short if long matches
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())
    
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    if ((0.042 < twoyr_vol) & (0.06 > twoyr_vol)): #only check long data if long data fits, save runtime
        short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
        twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
        twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
        twomo_avg = twomo_mid_data.mean() #calculate average 
        twomo_std_dev = twomo_mid_data.std() #calculate deviation
        twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    else:
        twomo_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [ ]:
tickers_0_1 = tickers[0:1000] #breaking it up into segments 

tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)

tickers_0_1_df = pd.concat([tickers_0_1[['Ticker']], tickers_0_1['PV'].apply(pd.Series)], axis=1)

print(tickers_0_1_df)

In [ ]:
tickers_1_2 = tickers[1000:2000] 

tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)

tickers_1_2_df = pd.concat([tickers_1_2[['Ticker']], tickers_1_2['PV'].apply(pd.Series)], axis=1)

print(tickers_1_2_df)

In [ ]:
tickers_2_3 = tickers[2000:3000] 

tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)

tickers_2_3_df = pd.concat([tickers_2_3[['Ticker']], tickers_2_3['PV'].apply(pd.Series)], axis=1)

print(tickers_2_3_df)

In [ ]:
tickers_3_end = tickers[3000:] 

tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)

tickers_3_end_df = pd.concat([tickers_3_end[['Ticker']], tickers_3_end['PV'].apply(pd.Series)], axis=1)

print(tickers_3_end_df)

In [ ]:
tickers_0_1_df_filtered = tickers_0_1_df[tickers_0_1_df["Short PV"].between(left = 0, right = 1)]
tickers_1_2_df_filtered = tickers_1_2_df[tickers_1_2_df["Short PV"].between(left = 0, right = 1)]
tickers_2_3_df_filtered = tickers_2_3_df[tickers_2_3_df["Short PV"].between(left = 0, right = 1)]
tickers_3_end_df_filtered = tickers_3_end_df[tickers_3_end_df["Short PV"].between(left = 0, right = 1)]

print(tickers_0_1_df_filtered)
print(tickers_1_2_df_filtered)
print(tickers_2_3_df_filtered)
print(tickers_3_end_df_filtered)

In [115]:
tickers_0_1 = tickers[0:1000] #breaking it up into segments 

tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(min_var_filter)

tickers_0_1_df = pd.concat([tickers_0_1[['Ticker']], tickers_0_1['PV'].apply(pd.Series)], axis=1)

print(tickers_0_1_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_25136/2088876059.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(min_var_filter)


    Ticker        PV
0       AA -1.000000
1      AAC  0.013234
2      AAN -1.000000
3      AAP -1.000000
4      AAT -1.000000
..     ...       ...
995    IQV -1.000000
996     IR  0.093885
997    IRM  0.062045
998    IRS -1.000000
999    IRT -1.000000

[1000 rows x 2 columns]


In [117]:
tickers_0_1_df_filtered = tickers_0_1_df[tickers_0_1_df["PV"].between(left = 0.03, right = 0.05)]
print(tickers_0_1_df_filtered)

    Ticker        PV
14      AC  0.046980
39     AFG  0.045881
75     ALL  0.048774
156    ATO  0.045061
224    BDJ  0.044632
261    BIT  0.046354
276    BLW  0.047735
280    BME  0.037044
284    BMY  0.042292
291    BOH  0.040808
310    BRW  0.043831
323    BTT  0.047707
339   BXMX  0.042960
393    CHE  0.044120
411     CL  0.039230
420    CLX  0.048755
550   DIAX  0.046080
566    DNP  0.037573
584    DSU  0.045038
608     ED  0.048417
632    ELV  0.042556
654    EPD  0.043098
657    EQC  0.047740
721    FEI  0.043625
729    FHN  0.042902
739    FLO  0.048547
798     GD  0.047102
817    GHI  0.038022
937    HSY  0.041217
963    IDA  0.043806
969    IGD  0.041143
